In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Customer_Churn_Prediction')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from split import split_data
from error_analysis import classification_summary
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Customer_Churn_Prediction/Telco-Customer-Churn-Final.csv")

In [ ]:

x = df.drop(columns="Churn", errors="ignore")
y = df["Churn"].astype(int)
x_train, x_val, x_test, y_train, y_val, y_test = split_data(x,y)

In [ ]:
to_scale = ['tenure','MonthlyCharges']
scaler = StandardScaler()

In [ ]:
for col in to_scale:
  x_train[col] = scaler.fit_transform(x_train[[col]])
  x_val[col] = scaler.transform(x_val[[col]])
  x_test[col] = scaler.transform(x_test[[col]])

In [ ]:
x_train

,SeniorCitizen,Partner,Dependents,tenure,InternetService,OnlineSecurity,TechSupport,PaperlessBilling,MonthlyCharges,Contract_Risk,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Automatic
4541,1,0,0,-1.102703,1,0,0,0,-0.592600,2,0,1,0
5709,0,0,0,-0.643016,0,0,0,0,-1.675908,1,0,1,0
7393,1,0,0,-1.102703,2,0,0,1,0.236599,2,1,0,0
10026,0,0,0,-0.977334,2,0,0,1,0.905102,2,1,0,0
3578,0,0,0,0.527097,1,1,0,0,-0.088211,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0,1,1,1.864369,1,1,1,1,0.754170,0,0,1,0
5191,0,1,1,-0.183329,1,1,1,1,0.813102,0,0,0,1
5390,1,0,0,-0.643016,2,0,0,1,1.102562,2,1,0,0
860,0,0,0,-0.057959,0,0,0,0,-1.658575,1,0,0,1


In [ ]:
model = Sequential([
    Input(shape=(x_train.shape[1],)),

    Dense(15, kernel_initializer="he_normal"),
    BatchNormalization(),
    Activation("relu"),
    Dropout(0.30),

    # Dense(10, kernel_initializer="he_normal"),
    # BatchNormalization(), Activation("relu"),
    # Dropout(0.25),

    Dense(1, activation="linear")
])

model.compile(loss=BinaryCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.fit(x_train, y_train, epochs=100,validation_data=(x_val, y_val))

Epoch 1/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6319 - val_loss: 0.4847
Epoch 2/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5089 - val_loss: 0.4556
Epoch 3/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4960 - val_loss: 0.4462
Epoch 4/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4745 - val_loss: 0.4352
Epoch 5/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4843 - val_loss: 0.4298
Epoch 6/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4722 - val_loss: 0.4280
Epoch 7/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4577 - val_loss: 0.4204
Epoch 8/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4566 - val_loss: 0.4141
Epoch 9/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4397 - val_loss: 0.4128
Epoch 10/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4537 - val_loss: 0.4062
Epoch 11/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4364 - val_loss: 0.4030
Epoch 12/100
259/259 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
val_logits = model.predict(x_val, batch_size=256, verbose=0).ravel()
val_prob = 1 / (1 + np.exp(-val_logits))
threshold = 0.5
pred = (val_prob >= threshold).astype(int)

In [ ]:
print(classification_summary(y_val, pred, val_prob))

{'confusion_matrix':           Pred_0  Pred_1
Actual_0     413      89
Actual_1      92     441, 'metrics': {'accuracy': 0.8251207729468599, 'precision': 0.8320754716981132, 'recall': 0.8273921200750469, 'f1': 0.8297271872060207, 'roc_auc': np.float64(0.9065333413064441)}, 'error_breakdown': {'TP': np.int64(441), 'FP': np.int64(89), 'FN': np.int64(92), 'TN': np.int64(413)}, 'classification_report': '              precision    recall  f1-score   support\n\n           0       0.82      0.82      0.82       502\n           1       0.83      0.83      0.83       533\n\n    accuracy                           0.83      1035\n   macro avg       0.82      0.83      0.82      1035\nweighted avg       0.83      0.83      0.83      1035\n'}
